# Validation of Local Heat Supply

In [2]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('../data/output/gee23_ST-min_NW-ref_2028_output_cleaned.csv')

# Calculate sums
local_heat_storage_discharging = df['local_heat_storage.heat_discharging'].sum()
excess_heat_discharging = df['local_heat_storage.excess_heat_discharging'].sum()
heat_grid_to_local = df['heat_grid.heat_grid_to_local'].sum()
local_heat_demand = df['local_heat_demand'].sum()

# Print results
print("Sums over entire period:")
print(f"Local heat storage discharging: {local_heat_storage_discharging:.2f} MWh")
print(f"Excess heat discharging: {excess_heat_discharging:.2f} MWh")
print(f"Heat grid to local: {heat_grid_to_local:.2f} MWh")
print(f"Local heat demand: {local_heat_demand:.2f} MWh")

# Calculate percentage of local heat demand covered by storage
storage_contribution = (local_heat_storage_discharging / local_heat_demand) * 100

print(f"\nPercentage of local heat demand covered by storage: {storage_contribution:.2f}%")

# Verify balance
total_supply = local_heat_storage_discharging + heat_grid_to_local
balance_check = abs(total_supply - local_heat_demand)
print(f"\nBalance check (should be close to 0): {balance_check:.2f} MWh")

KeyError: 'local_heat_storage.heat_discharging'